In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib widget

# Load style
plt.style.use('PlotStyle.mplstyle')
import matplotlib.colors as colors
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)

from datetime import datetime
from tqdm import tqdm

import os

# Load functions
import sys
sys.path.append("../../ExcessMortality")
import ExcessMortalityFunctions as emf
import AdditionalFunctions as ps


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)

saveFigures = True
saveFigures = False
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

In [ ]:
# Set paths
pathData = '../Data/'
pathResults = '../Data/AnalysisResults'
pathFigs = '../Figures/'

In [ ]:
# Flags and analysis parameters used in main analysis, set here to load correct results
numYears = 12 # Number of years on both sides of date to use for baseline calculations 
numYearsTot = (numYears*2) # The "name" of the baseline (i.e. +/- 5 years is a 10-year baseline, +/- 12 is a 24 year baseline)
thresholdExcess = 3 # Threshold (in terms of Z-scores) for identifying a day as having increased excess

# # Determine directory in which results was saved
# pathResultsUpper = pathResults + f'_Years{numYears}_Threshold{thresholdExcess}/'

# Additional parameters used
thresholdLower = 2 # Lower threshold used for determining the start and end of periods (in terms of Z-scores)
# maxDaysBelowThreshold = 4 # Number of days below thresholdLower before a period of excess is "stopped"
# minimumLengthOfEpidemic = 4 # Minimal number of days above thresholdExcess which is counted as a period of excess 
maxDaysBelowThreshold = 7 # Number of days below thresholdLower before a period of excess is "stopped"
minimumLengthOfEpidemic = 0 # Minimal number of days above thresholdExcess which is counted as a period of excess 
excessCountThreshold = 50 # Only save mortality crises with more than this number of excess deaths

# Various tests for sensitivity
# numYears = 6 # Number of years on both sides of date to use for baseline calculations 
# maxDaysBelowThreshold = 7 # Number of days below thresholdLower before a period of excess is "stopped"
# excessCountThreshold = 20 # Only save mortality crises with more than this number of excess deaths


# Determine filename to use for final results
# finalResultsFilename = 'AllCrises'+f'_Years{numYears}_Threshold{thresholdExcess}_LowerThreshold{thresholdLower}_MaxDaysBelow{maxDaysBelowThreshold}_minLength{minimumLengthOfEpidemic}_minCount{excessCountThreshold}'
finalResultsFilename = 'AllCrises'+f'_NonSmoothed_Years{numYears}_Threshold{thresholdExcess}_LowerThreshold{thresholdLower}_MaxDaysBelow{maxDaysBelowThreshold}_minLength{minimumLengthOfEpidemic}_minCount{excessCountThreshold}'
# finalResultsFilename = finalResultsFilename +'_Clustered'
finalResultsFilename = finalResultsFilename +'_Clustered_Grouped'
finalResultsFilename 

In [ ]:
# Load the table of results
dfCrises = pd.read_csv(pathData + finalResultsFilename + '.csv')
dfCrises['Start'] = pd.to_datetime(dfCrises['Start'])
dfCrises['End'] = pd.to_datetime(dfCrises['End'])
# dfCrises['DayWithMostBurials'] = pd.to_datetime(dfCrises['DayWithMostBurials'])
dfCrises['DayWithMostDeaths'] = pd.to_datetime(dfCrises['DayWithMostDeaths'])

# Make main overview figure

In [ ]:
import matplotlib.cm as cm
# Sort clusters (and name)

numClusters = len(dfCrises.Cluster.unique())
cmap = cm.get_cmap('jet',numClusters)
cmapBar = cm.get_cmap('rainbow')


In [ ]:
curClusters = np.arange(numClusters)
clusterNameDict = dict(zip(curClusters,[chr(65+x) for x in curClusters])) # chr(65) is 'A', chr(66) is 'B' ...

invMap = {v: k for k, v in clusterNameDict.items()}

In [ ]:
# Hand-picked order:
allAmts = [
'Staden København', 
'Københavns Amt',
'Frederiksborg Amt', 
'Holbæk Amt',
'Sorø Amt', 
'Præstø Amt', 
'Maribo Amt', 
'Bornholms Amt', 
'Odense Amt', 
'Svendborg Amt', 
'Hjørring Amt',
'Thisted Amt', 
'Ålborg Amt', 
'Viborg Amt',
'Ringkøbing Amt', 
'Ribe Amt',
'Randers Amt', 
'Skanderborg Amt', 
'Århus Amt', 
'Vejle Amt',
'Sønderborg Amt', 
'Haderslev Amt', 
'Åbenrå Amt', 
'Nordborg Amt',
'Tønder Amt',
]

In [ ]:
curAmt = allAmts[3]
curAmt

curdf = dfCrises[dfCrises.Amt == curAmt]
curdf.head()

In [ ]:
allAmtsNames = [x.replace(' Amt','').replace('Bornholms','Bornholm') for x in allAmts]
allAmtsNames[0] = 'Copenhagen, city'
allAmtsNames[1] = 'Copenhagen, county'

In [ ]:
# %matplotlib inline
# %matplotlib widget

In [ ]:
def sizeScaler(exc):
    return 5*np.log(exc/50 + 4) 
    # return 3*np.sqrt(exc/50-0.4) 

fig,ax = plt.subplots(figsize=(20,10))


# for amtNum,curAmt in enumerate(allAmts[:5]):
# for amtNum,curAmt in enumerate(allAmts):
for amtNum,curAmt in enumerate(allAmts[::-1]):
    curdf = dfCrises[dfCrises.Amt == curAmt].sort_values('Start')
    for i,r in curdf.iterrows():

        # curSize = 10
        # curSize = 2*np.log(r.Excess)
        # curSize = 5*np.log(r.Excess/50 + 4) 
        curSize = sizeScaler(r.Excess)

        curHeight = i
        curHeight = amtNum

        curColor = cmap(invMap[r.Cluster]/numClusters)

        ax.plot([r.Start,r.End],[curHeight,curHeight],color=curColor,lw=curSize, solid_capstyle='round')

ax.set_yticks(np.arange(len(allAmts)))
ax.set_ylim([-0.5,len(allAmts)])
# ax.set_yticklabels(allAmts)
# ax.set_yticklabels(allAmts[::-1])
ax.set_yticklabels(allAmtsNames[::-1])

ax.xaxis.tick_top()
ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True) 
# ax.spines['left'].set_visible(False)
# ax.spines['bottom'].set_visible(False)

# ax.set_xticks(np.arange(
#     np.datetime64('1810'),
#     np.datetime64('1921'),
#     np.timedelta64(10,'Y')
# ))

ax.tick_params(axis='x',width=1, length=6)
ax.tick_params(axis='y',color='xkcd:grey')

import matplotlib.dates as mdates 
ax.xaxis.set_major_locator(mdates.YearLocator(10))
ax.xaxis.set_minor_locator(mdates.YearLocator(1))

from matplotlib.dates import DateFormatter
ax.xaxis.set_major_formatter(DateFormatter('%Y'))

ax.set_xlim(right=np.datetime64('1916'))



curExc = 100
curSize = sizeScaler(curExc)
for i in range(numClusters):
    
    curColor = cmap(i/numClusters)
    curLabel = 'Cluster '+clusterNameDict[i]
    ax.plot([20,20],[30,30],color=curColor,lw=curSize, solid_capstyle='round',label=curLabel)

curExc = 50
curSize = sizeScaler(curExc)
ax.plot([20,20],[30,30],color='k',lw=curSize, solid_capstyle='round',label=f'{curExc:10.0f}')
curExc = 100
curSize = sizeScaler(curExc)
ax.plot([20,20],[30,30],color='k',lw=curSize, solid_capstyle='round',label=f'{curExc:10.0f}')
curExc = 250
curSize = sizeScaler(curExc)
ax.plot([20,20],[30,30],color='k',lw=curSize, solid_capstyle='round',label=f'{curExc:10.0f}')
curExc = 1000
curSize = sizeScaler(curExc)
ax.plot([20,20],[30,30],color='k',lw=curSize, solid_capstyle='round',label=f'{curExc:10.0f}')
curExc = 3000
curSize = sizeScaler(curExc)
ax.plot([20,20],[30,30],color='k',lw=curSize, solid_capstyle='round',label=f'{curExc:10.0f}')


ax.legend(bbox_to_anchor=(1.01,0.5),loc='center left',handlelength=0.5,labelspacing=2,fontsize=12,borderpad=2)

ax.grid(True,axis='y',color='xkcd:light grey')
ax.grid(True,axis='x',color=(0.95,0.95,0.95,0.6))

fig.tight_layout()

if saveFigures:
    fig.savefig(pathFigs+'OverviewOfCrises')

In [ ]:
# # Test of scaling
# def sizeScaler(exc):
#     return 5*np.log(exc/50 + 4) 
#     # return 3*np.sqrt(exc/50-0.4) 

# fig,ax = plt.subplots()

# valsToCheck = [50,100,200,500,1000,2000]
# for i,va in enumerate(valsToCheck):
#     # curWi = 4*np.log(va/50 + 1) 
    
#     curSize = 5*np.log(va/50 + 4) 
#     curSize = 4*np.sqrt(va/50 -0.2) 

#     ax.plot([1,10],[i,i],lw=curSize)